In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [2]:
layer = tf.keras.layers
print("tensorflow version check : ", tf.__version__)
print("gpu check : ", tf.test.is_gpu_available())

tensorflow version check :  2.0.0-beta1
gpu check :  True


In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [4]:
import pathlib
data_dir = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', untar=True)
data_dir = pathlib.Path(data_dir)

In [5]:
image_count = len(list(data_dir.glob('*/*.jpg')))
image_count

3670

In [6]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
CLASS_NAMES

array(['roses', 'tulips', 'dandelion', 'daisy', 'sunflowers'],
      dtype='<U10')

# 1. get dataset on memory

In [7]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))

In [8]:
for f in list_ds.take(5):
  print(f.numpy())

b'/home/dan/.keras/datasets/flower_photos/dandelion/8747223572_dcd9601e99.jpg'
b'/home/dan/.keras/datasets/flower_photos/roses/15190665092_5c1c37a066_m.jpg'
b'/home/dan/.keras/datasets/flower_photos/dandelion/3688128868_031e7b53e1_n.jpg'
b'/home/dan/.keras/datasets/flower_photos/dandelion/425800274_27dba84fac_n.jpg'
b'/home/dan/.keras/datasets/flower_photos/daisy/6884975451_c74f445d69_m.jpg'


# data processing pipeline

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

In [ ]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, '/')
  # The second to last is the class-directory
  return parts[-2] == CLASS_NAMES

In [ ]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

In [ ]:
def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [ ]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
for image, label in labeled_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

In [ ]:
import numpy as np

In [ ]:
c = np.array(['a', 'b', 'c'])

In [ ]:
'a' == c

In [ ]:
tf.constant(b'daisy', shape=(), dtype=tf.string) == CLASS_NAMES

In [ ]:
cifar100 = tf.keras.datasets.cifar100
train, test = cifar100.load_data(label_mode='coarse')
train_ds = tf.data.Dataset.from_tensor_slices(
    (train[0], train[1])).shuffle(5000).batch(64)
test_ds = tf.data.Dataset.from_tensor_slices(
    (test[0], test[1])).shuffle(5000).batch(64)

In [ ]:
IMG_SIZE = 128
LEARNING_RATE = 0.0001
fine_tune_at = 172

# 2. Add additional processing into your dataset

In [ ]:
def parse_image(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image, label


train_ds = train_ds.map(parse_image)
test_ds = test_ds.map(parse_image)
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# 3. Create the base model from the pre-trained convnets

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                            include_top=False,
                                            weights='imagenet')

### 3.1 Freeze the convolutional base and use it as feature extractor

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))
base_model.trainable = False

### 3.2 finetune the part of base model

In [ ]:
base_model.trainable = True
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))
# Fine tune from this layer onwards
# Freeze all the layers before the `fine_tune_at` layer

for l in base_model.layers[:fine_tune_at]:
    print(l.trainable)
    l.trainable = False
    print('now trainable is false')

# 4. model redefinition and training

In [ ]:
inputs = keras.Input(shape=(128, 128, 3))
feature_batch = base_model(inputs)
global_average_layer = layer.Flatten()(feature_batch)
prediction = layer.Dense(100, activation='softmax')(global_average_layer)
model = keras.Model(inputs, prediction)

In [ ]:
model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [ ]:
print(len(model.trainable_variables))

print('========================Training===============================')

model.fit(train_ds, epochs=10, validation_data=test_ds)

# print('========================Evaluation===============================')
#
# test_loss, test_acc = model.evaluate(test_ds)

del model
keras.backend.clear_session()

https://benchmarks.ai/cifar-100

In [ ]:
del model
keras.backend.clear_session()